## Analysis of the Distribution of the Correct Hits ##

In [ ]:
import os
import pandas as pd
import numpy as np
import math

%matplotlib inline
import matplotlib.pyplot as plt
from plotly import __version__
print(__version__)

import plotly.express as px
import plotly.graph_objects as go
from plotly.colors import n_colors
from plotly.subplots import make_subplots


from plotly.offline import download_plotlyjs, init_notebook_mode,plot,iplot
init_notebook_mode(connected=True)  #connect with Javascript

import numpy as np
import scipy.special

list_protocols="""
'mixed_ali-delim-delim',
'single_pep-delim-delim',
'unpaired_ali-delim-delim',
'mixed_ali-fl-fl',
'mixed_ali-delim-fl',
'unpaired_ali-delim-fl',
'mixed_ali-delim-200',
'mixed_ali-delim-100',
'single_pep-delim-100',
'single_pep-delim-200'
"""


### Recover the environment variable in the config file ###

In [ ]:
import configparser

#script_path = os.path.abspath(__file__)
script_dir = "../" #os.path.dirname(os.path.dirname(script_path))

config = configparser.ConfigParser()
config.read(os.path.join(f'{script_dir}', 'config.ini'))

WORKING_DIR = os.path.abspath("../..") #config['DEFAULT']['WORKING_DIR']""
DATA_DIR = os.path.abspath(os.path.join(WORKING_DIR, 'data', ))


CUTMODELS_DIR = os.path.abspath(os.path.join(DATA_DIR, config['DEFAULT']['CUTMODELS_DIR']))
CAPRIEVAL_DIR = os.path.abspath(os.path.join(DATA_DIR, config['DEFAULT']['CAPRIEVAL_DIR']))
REFERENCE_STRUCT_DIR = os.path.abspath(os.path.join(DATA_DIR, config['DEFAULT']['REFERENCE_DIR']))

n_iterations = int(config['DEFAULT']['N_ITERATIONS'])

RESULTS_DIR = os.path.abspath(os.path.join(DATA_DIR, config['DEFAULT']['RESULTS_DIR']))
fglobaloutput_path = os.path.abspath(os.path.join(RESULTS_DIR, config['DEFAULT']['OUTPUT_GLOBAL']))

In [ ]:
path = RESULTS_DIR
path_figures = os.path.join(path, 'Figures')
if not os.path.isdir(path_figures):
    os.system(f"mkdir {path_figures}")
DO_DUMP_FIGURES = False

Ncol = 23
path2file = fglobaloutput_path
extension =''

l_conditions = ["mixed_ali-delim-delim","single_pep-delim-delim","unpaired_ali-delim-delim"]

Ncol = 7 # How many items (columns) in the file
l_df = []
for cond in l_conditions:
    result_file = rf"Stats_5iterations{extension}_{cond}.out"
    path2file = os.path.join(path,result_file)
    list_header = []
    with open(path2file) as fin:
        f = fin.readlines()[:Ncol]
        for l in f:
            s = l.split()[-1]
            colname = s.split(':')[1]
            list_header.append(colname)
    df = pd.read_table(path2file, sep="\t", header = None, names = list_header, skiprows=Ncol, low_memory=False)
    l_df.append(df)

df = pd.concat(l_df)

df

### Method for plotting scatter plots ###

In [ ]:
from sklearn.linear_model import LinearRegression

def plot_scatter(df, color1='rgb(100,100,100)', color2='rgb(250,0,0)',):
    #
    # Figure Plotting with Scatter Points representation
    #

    fig = go.Figure()
    fig.update_layout(
        autosize=True,
        width=600,
        height=500,
    )

    fig.update_layout(

        plot_bgcolor = "rgba(0,0,0,0)",
        xaxis_title="AF2 Combined Score",
        yaxis_title="DockQ Score",
        legend_title="Values",
        font=dict(
            family="Arial",
            size=18,
            color="black"
        ),
        legend=dict(
        orientation="h",
        yanchor="bottom",
        y=1.02,
        xanchor="right",
        x=0.7,
        ),
    )
    fig.update_xaxes(range=[0.0, 1.0])
    fig.update_yaxes(range=[0.0, 1.0])

    fig.update_xaxes(showline=True, linewidth=2, linecolor='black', mirror=True, 
        showgrid=True,
        ticks="outside",
        tickson="boundaries",
        ticklen=5)
    fig.update_yaxes(showline=True, linewidth=2, linecolor='black', mirror=True, 
        showgrid=True,
        ticks="outside",
        tickson="boundaries",
        ticklen=5)

    hovertext = list()
    for yi, yy in enumerate(df['Path2File']):
        index_pdb = df['index_pdb'].values[yi]
        capri_rank = df['CAPRIrankPeptide'].values[yi]
        irms = df['IRMS'].values[yi]
        filename = os.path.basename(df['Path2File'].values[yi])
        txt = f"index_pdb: {index_pdb}<br />capri_rank: {capri_rank}<br />iRMS: {irms:.1f}<br />file: {filename}<br />"
        hovertext.append(txt)

    # Ranges if color gradient of AF2 scores
    #l_rangecat = ['0.0-0.2', '0.2-0.4', '0.4-0.6', '0.6-0.8', '0.8-1.0']
    # Ranges if color gradient of DockQ scores
    l_rangecat = ['0.0-0.23', '0.23-0.49', '0.49-0.8', '0.8-1.0']

    colors = n_colors(color1, color2, len(l_rangecat), colortype='rgb')
    #colors[0] = 'rgb(100,100,100)'
    fig.add_trace(go.Scatter(
        x=df['CombinedSCORE'],
        y=df['DockQ'],
        name='DockQ vs AF2 Combined Score',
        marker=dict(
        size=4,
        color=df['DockQ'], #set color equal to a variable
        colorscale=colors, # one of plotly colorscales
        showscale=False,
        line=dict(width=1,color='rgb(100, 100, 100)')
        ),
        #marker_color='rgb(50, 100, 200)',
        mode='markers',
        text=hovertext,
        hoverinfo=f'text',
    ))
    

    return fig


### Method for plotting violin plots ###

In [ ]:
def plot_violin(df):
    #
    # Figure Plotting with Violin representation
    #

    fig = go.Figure()
    fig.update_layout(
        autosize=True,
        width=600,
        height=500,
        font=dict(
            family="Arial",
            size=18,
            #color="RebeccaPurple"
        ),
    )

    fig.update_layout(
        plot_bgcolor = "rgba(0,0,0,0)",
        xaxis_title="AF2 Combined Score",
        yaxis_title="DockQ Score",
    )
    fig.update_xaxes(showline=True, linewidth=2, linecolor='black', mirror=True, 
        showgrid=True,
        ticks="outside",
        tickson="boundaries",
        ticklen=5)
    fig.update_yaxes(showline=True, linewidth=2, linecolor='black', mirror = True, 
        showgrid=True,
        ticks="outside",
        tickson="boundaries",
        ticklen=5)

    fig.update_yaxes(range=[-0.2, 1.2])

    ranges = [(0,0.2), (0.2,0.4), (0.4,0.6), (0.6, 0.8), (0.8, 1.0)]
    # Ranges if color gradient of AF2 scores
    #l_rangecat = ['0.0-0.2', '0.2-0.4', '0.4-0.6', '0.6-0.8', '0.8-1.0']
    # Ranges if color gradient of DockQ scores
    l_rangecat = ['0.0-0.23', '0.23-0.49', '0.49-0.8', '0.8-1.0']
    l_values = []
    for sc in df['CombinedSCORE'].values:
        cat = l_rangecat[int(math.floor(sc*5))]
        l_values.append(cat)
    df['ScoreCategory'] = l_values

    colors = n_colors('rgb(100,100,100)', 'rgb(250,0,0)', len(l_rangecat), colortype='rgb')
    
    for range, color in zip(l_rangecat,colors):
        fig.add_trace(go.Violin(x=df['ScoreCategory'][df['ScoreCategory'] == range],
                                y=df['DockQ'][df['ScoreCategory'] == range],
                                line_color=color,
                                name=range,
                                box_visible=True,
                                meanline_visible=True))


    fig.update_layout(yaxis_zeroline=True)
    return fig

### Method for plotting Box plots ###

In [ ]:
def plot_box(df, color1='rgb(100,100,100)', color2='rgb(250,0,0)',):
    #
    # Figure Plotting with Violin representation
    #

    fig = go.Figure()
    fig.update_layout(
        autosize=True,
        width=750,
        height=500,
        font=dict(
            family="Arial",
            size=18,
            color="black"
        ),
    )

    fig.update_layout(
        plot_bgcolor = "rgba(0,0,0,0)",
        xaxis_title="AF2 Combined Score",
        yaxis_title="CAPRI Peptide Rank", 
    )
    fig.update_xaxes(showline=True, linewidth=2, linecolor='black', mirror=True, 
        showgrid=True,
        ticks="outside",
        tickson="boundaries",
        ticklen=5)
    fig.update_yaxes(showline=True, linewidth=2, linecolor='black', mirror = True, 
        showgrid=True,
        ticks="outside",
        tickson="boundaries",
        ticklen=5)

    #ranges = [(0,0.2), (0.2,0.4), (0.4,0.6), (0.6, 0.8), (0.8, 1.0)]
    ranges = [(0,0.23), (0.23,0.49), (0.49,0.8), (0.8, 1.0)]
    # Ranges if color gradient of AF2 scores
    #l_rangecat = ['0.0-0.2', '0.2-0.4', '0.4-0.6', '0.6-0.8', '0.8-1.0']
    # Ranges if color gradient of DockQ scores
    l_rangecat = ['0.0-0.23', '0.23-0.49', '0.49-0.8', '0.8-1.0']
    l_CAPRI = ['Incorrect','Acceptable','Medium','High']
    d_range2CAPRI = {zip(l_rangecat,l_CAPRI)}
    l_values = []
    #for sc in df['CombinedSCORE'].values:
    #for sc in df['DockQ'].values:
    for sc in df['CAPRIrankPeptide'].values:
        #for ii,r in enumerate(ranges):
        for ii,r in enumerate(l_CAPRI):
            #if sc >= r[0] and sc < r[1]:
            if sc == r:
                cat = l_rangecat[ii]
        l_values.append(cat)
    df['ScoreCategory'] = l_values

    colors = n_colors(color1, color2, len(l_rangecat), colortype='rgb')
    fcolors = colors
    colors[0] = 'rgb(0,0,0)'
    fcolors[0] = 'rgb(255,255,255)'
    for rangeval, color, fcolor in zip(l_CAPRI,colors,fcolors):
        fig.add_trace(go.Box(y = df['CAPRIrankPeptide'][df['CAPRIrankPeptide'] == rangeval],
                                #y=df['DockQ'][df['ScoreCategory'] == range],
                                x = df['CombinedSCORE'][df['CAPRIrankPeptide'] == rangeval],
                                line_color = 'black',
                                fillcolor = fcolor,
                                marker_size=4,
                                name = rangeval,
                                line_width=1
                            ))

    fig.update_traces(orientation='h') # horizontal box plots
    #fig.update_yaxes(range=[-0.2, 1.2])
    fig.update_xaxes(range=[0, 1],
                    dtick=0.2)

    #fig.update_layout(yaxis_zeroline=True)
    return fig



### Plotting of the scores ###

In [ ]:
#
# IF NEEDED : Which PDB to exclude :
#  add to list_exclude elements as <index>_<PDB> 
#  Typically better to exclude PDBs involving trimers since the ipTMscore is not properly calculated for N_multimers > 2

list_exclude = []


df_with_index = df.set_index("index_pdb")
df = df_with_index.drop(list_exclude)
df = df.reset_index()

In [ ]:
list_entries_sorted = [y[1] for y in sorted([[int(x.split('_')[0]), x] for x in list(set(df['index_pdb']))])]
print(list_entries_sorted)
list_conditions = [x for x in list(set(df['protocol']))]
print(list_conditions)

In [ ]:
# Scatter plot of all the models

fig = plot_scatter(df,
               color1='rgb(255,255,255)', 
               color2='rgb(0,0,0)'
              )

X = df['CombinedSCORE'].values.reshape(-1, 1) # reshape as a column vector
model = LinearRegression()
model.fit(X, df['DockQ'])
pearson_corr = math.sqrt(model.score(X, df['DockQ']))
print(f'Pearsons correlation: {pearson_corr:.2f}')
x_range = np.linspace(X.min(), X.max(), 100)
#x_range = np.linspace(0, 1, 100)
y_range = model.predict(x_range.reshape(-1, 1))
fig.add_trace(go.Scatter(x=x_range, y=y_range, line_color='black', showlegend=False))

if DO_DUMP_FIGURES:
    fig.write_image(os.path.join(path,"Figures", f"DockQ_vs_AF2_{'-'.join(list_conditions)}_ScatterPlot_nocolor.svg"))

fig.show()

In [ ]:
from scipy.stats import pearsonr
from scipy.stats import spearmanr

x=df['CombinedSCORE']
y=df['DockQ']
# calculate Pearson's correlation
corr, _ = pearsonr(x, y)
print('Pearsons correlation: %.2f' % corr)

# calculate spearman's correlation
corrsp, _ = spearmanr(x, y)
print('Spearmans correlation: %.2f' % corrsp)

In [ ]:
# Box plot of all the models

fig = plot_box(df,
               color1='rgb(255,255,255)', 
               color2='rgb(0,0,0)'
              )
df1 = df
fig.write_image(os.path.join(path,"Figures", f"DockQ_vs_AF2_{'-'.join(list_conditions)}_BoxPlot_nocolor.svg"))
fig.show()
